### Code for the blog **[How to use nested data types effectively in SQL](https://www.startdataengineering.com/post/use-structs-sql/)**

# [SETUP] 

In [2]:
! python ./setup.py

Cleaning up (if any existing) tpch db file tpch.db
Creating TPCH input data at tpch.db


## Connect to DuckDB

In [3]:
import duckdb
import pandas as pd

%load_ext sql
conn = duckdb.connect("tpch.db")
%sql conn --alias duckdb

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [17]:
%%sql
show tables;

Running query in 'duckdb'

+-------------+
|     name    |
+-------------+
|   customer  |
|   lineitem  |
|    nation   |
|    orders   |
|     part    |
|   partsupp  |
|    region   |
|   supplier  |
| wide_orders |
+-------------+

# [Data Model]

The TPC-H data represents a car parts seller’s data warehouse, where we record orders, items that make up that order (lineitem), supplier, customer, part (parts sold), region, nation, and partsupp (parts supplier). 

Note: Have a copy of the data model as you follow along; this will help in understanding the examples provided and in answering exercise questions.

![](../../tpch_erd.png)


# Using nested data types effectively

##  Use STRUCT for one-to-one & hierarchical relationships

In [ ]:
%%sql
-- without nested data type
SELECT l.*,
c.*,
s.*
FROM 
    lineitem l
LEFT JOIN 
    orders o ON l.l_orderkey = o.o_orderkey
LEFT JOIN 
    customer c ON o.o_custkey = c.c_custkey
LEFT JOIN 
    supplier s ON l.l_suppkey = s.s_suppkey
LIMIT 5;

![Cols to struct](./images/col-struct.png)

In [ ]:
%%sql
-- with nested data types
SELECT 
    l.*, 
    struct_pack(
        id := c.c_custkey,
        name := c.c_name,
        address := c.c_address,
        nationkey := c.c_nationkey,
        phone := c.c_phone,
        acctbal := c.c_acctbal,
        mktsegment := c.c_mktsegment,
        comment := c.c_comment
    ) AS customer,
    struct_pack(
        id := s.s_suppkey,
        name := s.s_name,
        address := s.s_address,
        nationkey := s.s_nationkey,
        phone := s.s_phone,
        acctbal := s.s_acctbal,
        comment := s.s_comment
    ) AS supplier
FROM 
    lineitem l
LEFT JOIN 
    orders o ON l.l_orderkey = o.o_orderkey
LEFT JOIN 
    customer c ON o.o_custkey = c.c_custkey
LEFT JOIN 
    supplier s ON l.l_suppkey = s.s_suppkey
LIMIT 5;

In [ ]:
%%sql
-- Hierarchical data 
SELECT 
    l.*, 
    struct_pack(
        id := c.c_custkey,
        name := c.c_name,
        address := c.c_address,
        nationkey := c.c_nationkey,
        phone := c.c_phone,
        acctbal := c.c_acctbal,
        mktsegment := c.c_mktsegment,
        comment := c.c_comment,
        nation := struct_pack(
            nationkey := n.n_nationkey,
            name := n.n_name,
            regionkey := n.n_regionkey,
            comment := n.n_comment
        )
    ) AS customer,
    struct_pack(
        id := s.s_suppkey,
        name := s.s_name,
        address := s.s_address,
        nationkey := s.s_nationkey,
        phone := s.s_phone,
        acctbal := s.s_acctbal,
        comment := s.s_comment,
        nation := struct_pack(
            nationkey := sn.n_nationkey,
            name := sn.n_name,
            regionkey := sn.n_regionkey,
            comment := sn.n_comment
            )
    ) AS supplier
FROM 
    lineitem l
LEFT JOIN 
    orders o ON l.l_orderkey = o.o_orderkey
LEFT JOIN 
    customer c ON o.o_custkey = c.c_custkey
LEFT JOIN
    nation n ON c.c_nationkey = n.n_nationkey
LEFT JOIN 
    supplier s ON l.l_suppkey = s.s_suppkey
LEFT JOIN
    nation sn ON s.s_nationkey = sn.n_nationkey
LIMIT 5;

In [ ]:
%%sql
# exercise, to the above query add region attributes
SELECT l.*,
    struct_pack(
        id := c.c_custkey,
        name := c.c_name,
        address := c.c_address,
        nationkey := c.c_nationkey,
        phone := c.c_phone,
        acctbal := c.c_acctbal,
        mktsegment := c.c_mktsegment,
        comment := c.c_comment,
        nation := struct_pack(
            nationkey := n.n_nationkey,
            name := n.n_name,
            regionkey := n.n_regionkey,
            comment := n.n_comment,
            region := struct_pack(
                regionkey := r.r_regionkey,
                name := r.r_name,
                comment := r.r_comment
            )
        )
    ) AS customer,
    struct_pack(
        id := s.s_suppkey,
        name := s.s_name,
        address := s.s_address,
        nationkey := s.s_nationkey,
        phone := s.s_phone,
        acctbal := s.s_acctbal,
        comment := s.s_comment,
        nation := struct_pack(
            nationkey := sn.n_nationkey,
            name := sn.n_name,
            regionkey := sn.n_regionkey,
            comment := sn.n_comment,
            region := struct_pack(
                regionkey := sr.r_regionkey,
                name := sr.r_name,
                comment := sr.r_comment
            )
        )
    ) AS supplier
FROM 
    lineitem l
LEFT JOIN 
    orders o ON l.l_orderkey = o.o_orderkey
LEFT JOIN 
    customer c ON o.o_custkey = c.c_custkey
LEFT JOIN
    nation n ON c.c_nationkey = n.n_nationkey
LEFT JOIN
    region r ON n.n_regionkey = r.r_regionkey
LEFT JOIN 
    supplier s ON l.l_suppkey = s.s_suppkey
LEFT JOIN
    nation sn ON s.s_nationkey = sn.n_nationkey
LEFT JOIN
    region sr ON sn.n_regionkey = sr.r_regionkey
LIMIT 5;

## Use ARRAY[STRUCT] for one-to-many relationships

![Rows to List](./images/row-list.png)

In [ ]:
%%sql
WITH line_items as (
SELECT 
    l_orderkey as orderkey,
    array_agg(struct_pack(
        lineitemkey := l.l_linenumber,
        partkey := l.l_partkey,
        suppkey := l.l_suppkey,
        quantity := l.l_quantity,
        extendedprice := l.l_extendedprice,
        discount := l.l_discount,
        tax := l.l_tax,
        returnflag := l.l_returnflag,
        linestatus := l.l_linestatus,
        shipdate := l.l_shipdate,
        commitdate := l.l_commitdate,
        receiptdate := l.l_receiptdate,
        shipinstruct := l.l_shipinstruct,
        shipmode := l.l_shipmode,
        comment := l.l_comment
    )) AS lineitems
FROM 
    lineitem l 
GROUP BY 
    l_orderkey)
SELECT o.*,
    LENGTH(l.lineitems) as num_lineitems, -- count number of elements in array
    l.lineitems
FROM orders o
LEFT JOIN line_items l
    ON o.o_orderkey = l.orderkey
LIMIT 5;

## Using nested data types in data processing

In [5]:
%%sql
DROP TABLE IF EXISTS wide_orders;

Running query in 'duckdb'

+---------+
| Success |
+---------+
+---------+

In [6]:
%%sql
CREATE TABLE IF NOT EXISTS wide_orders AS 
WITH line_items as (
SELECT 
    l_orderkey as orderkey,
    array_agg(struct_pack(
        lineitemkey := l.l_linenumber,
        partkey := l.l_partkey,
        suppkey := l.l_suppkey,
        quantity := l.l_quantity,
        extendedprice := l.l_extendedprice,
        discount := l.l_discount,
        tax := l.l_tax,
        returnflag := l.l_returnflag,
        linestatus := l.l_linestatus,
        shipdate := l.l_shipdate,
        commitdate := l.l_commitdate,
        receiptdate := l.l_receiptdate,
        shipinstruct := l.l_shipinstruct,
        shipmode := l.l_shipmode,
        comment := l.l_comment
    )) AS lineitems
FROM 
    lineitem l 
GROUP BY 
    l_orderkey)
SELECT 
    o.*,
    l.lineitems,
    struct_pack(
        id := c.c_custkey,
        name := c.c_name,
        address := c.c_address,
        nationkey := c.c_nationkey,
        phone := c.c_phone,
        acctbal := c.c_acctbal,
        mktsegment := c.c_mktsegment,
        comment := c.c_comment,
        nation := struct_pack(
            nationkey := n.n_nationkey,
            name := n.n_name,
            regionkey := n.n_regionkey,
            comment := n.n_comment
        )
    ) AS customer
FROM 
    orders o
LEFT JOIN 
    line_items l ON o.o_orderkey = l.orderkey
LEFT JOIN 
    customer c ON o.o_custkey = c.c_custkey
LEFT JOIN
    nation n ON c.c_nationkey = n.n_nationkey;

Running query in 'duckdb'

+-------+
| Count |
+-------+
| 15000 |
+-------+

### STRUCT enables more straightforward data schema and data access

In [ ]:
%%sql
SELECT o_orderkey,
    customer.name,
    customer.address,
    lineitems[1] as first_lineitem
FROM wide_orders
limit 2;

### Nested data types can be sorted



In [ ]:
%%sql
SELECT customer
FROM wide_orders 
GROUP BY 1
ORDER BY 1
LIMIT 5

In [ ]:
%%sql
# exercise: Sort ARRAY of lineitems, how is it sorted? What do you think is the sort order based on?
SELECT *, 
    array_sort(lineitems) AS sorted_lineitems
FROM wide_orders
LIMIT 5;

### UNNEST ARRAY to rows and GROUP rows to ARRAY

In [12]:
%%sql
-- rows to ARRAY
WITH lineitems as (SELECT 
    o.o_orderkey,
    UNNEST(o.lineitems) as line_item
FROM 
    wide_orders o),
unnested_line_items AS (
SELECT o_orderkey,
    line_item.lineitemkey,
    line_item.partkey,
    line_item.quantity
    FROM lineitems
)
SELECT o_orderkey,
array_agg(struct_pack(
        line_item_key := lineitemkey,
        part_key := partkey,
        quantity := quantity)) as lineitems
FROM unnested_line_items
GROUP BY 1
LIMIT 5;

Running query in 'duckdb'

+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| o_orderkey |                                                                                                                                                                                                                                               lineitems                                                                                                                                                                                                                                                |
+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|     2      |                                                                                                                                                                                                                 [{'line_item_key': 1, 'part_key': 1062, 'quantity': Decimal('38.00')}]                                                                                                                                                                                                                 |
|     4      |                                                                                                                                                                                                                 [{'line_item_key': 1, 'part_key': 881, 'quantity': Decimal('30.00')}]                                                                                                                                                                                                                  |
|     7      | [{'line_item_key': 1, 'part_key': 1821, 'quantity': Decimal('12.00')}, {'line_item_key': 2, 'part_key': 1453, 'quantity': Decimal('9.00')}, {'line_item_key': 3, 'part_key': 948, 'quantity': Decimal('46.00')}, {'line_item_key': 4, 'part_key': 1631, 'quantity': Decimal('28.00')}, {'line_item_key': 5, 'part_key': 1519, 'quantity': Decimal('38.00')}, {'line_item_key': 6, 'part_key': 793, 'quantity': Decimal('35.00')}, {'line_item_key': 7, 'part_key': 1573, 'quantity': Decimal('5.00')}] |
|     34     |                                                                                                                                            [{'line_item_key': 1, 'part_key': 884, 'quantity': Decimal('13.00')}, {'line_item_key': 2, 'part_key': 895, 'quantity': Decimal('22.00')}, {'line_item_key': 3, 'part_key': 1696, 'quantity': Decimal('6.00')}]                                                                                                                                             |
|     37     |                                                                                                                                            [{'line_item_key': 1, 'part_key': 227, 'quantity': Decimal('40.00')}, {'line_item_key': 2, 'part_key': 1268, 'quantity': Decimal('39.00')}, {'line_item_key': 3, 'part_key': 130, 'quantity': Decimal('43.00')}]                                                                                                              

In [11]:
%%sql
-- ARRAY to rows
WITH lineitems AS (
    SELECT
      o.o_orderkey,
      UNNEST (o.lineitems) AS line_item
    FROM
      wide_orders o
)
SELECT
  o_orderkey,
  line_item.lineitemkey,
  line_item.partkey,
  line_item.quantity
FROM
  lineitems
LIMIT
  5

Running query in 'duckdb'

+------------+-------------+---------+----------+
| o_orderkey | lineitemkey | partkey | quantity |
+------------+-------------+---------+----------+
|     1      |      1      |   1552  |  17.00   |
|     1      |      2      |   674   |  36.00   |
|     1      |      3      |   637   |   8.00   |
|     1      |      4      |    22   |  28.00   |
|     1      |      5      |   241   |  24.00   |
+------------+-------------+---------+----------+

### Improve OBT usability with nested data types

![Nested OBT to pre aggregate](./images/nested-preagg.png)

In [ ]:
%%sql
select *
FROM wide_orders
limit 1

In [ ]:
%%sql
-- get lineitem metrics, with array aggregation functions (specific to DuckDB)
SELECT o_orderkey,
    o_totalprice AS order_total_price,
    len(lineitems) as num_line_items,
    array_reduce(array_transform(lineitems, x -> x.quantity), (x, y) -> x + y) as total_line_item_quantity
    -- array_transform take only quantity field of each element of array
FROM wide_orders
ORDER BY
  1
LIMIT
  10;

In [ ]:
%%sql
-- Get order metrics
SELECT
  o_orderdate,
  SUM(o_totalprice) AS order_total_price
FROM
  wide_orders
GROUP BY 1
ORDER BY 1
LIMIT 5;

In [ ]:
%%sql
-- get lineitem metrics
WITH
  lineitems AS (
    SELECT
      o.o_orderkey,
      UNNEST (o.lineitems) AS line_item
    FROM
      wide_orders o
  )
SELECT
  o_orderkey,
  COUNT(line_item.lineitemkey) AS num_line_items,
  SUM(line_item.quantity) AS total_line_item_quantity
FROM
  lineitems
GROUP BY
  1
ORDER BY
  1
LIMIT
  10;

In [ ]:
# exercise: create a wide_orders_v2 table that combines all the tables in the TPCH data model


## Ensure your performance meets your expectations

In [ ]:
# unnest is an expensive opeation
result = %sql EXPLAIN WITH lineitems AS ( SELECT o.o_orderkey, UNNEST (o.lineitems) AS line_item FROM wide_orders o ) SELECT o_orderkey, COUNT(line_item.lineitemkey) AS num_line_items, SUM(line_item.quantity) AS total_line_item_quantity FROM lineitems GROUP BY 1 ORDER BY 1
print(result)

In [ ]:
result = %sql EXPLAIN SELECT o_orderkey, o_totalprice AS order_total_price, len(lineitems) as num_line_items, array_reduce(array_transform(lineitems, x -> x.quantity), (x, y) -> x + y) as total_line_item_quantity FROM wide_orders ORDER BY 1 LIMIT 10;
print(result)